In [1]:
%load_ext lab_black
%load_ext autoreload
%autoreload 2

# This is analyze the fit function of the accelerated DBSCAN

In [45]:
import numpy as np
import math
import pandas as pd
import itertools
import copy

In [46]:
max_number_of_tracks = 232
max_number_of_tracks_power_2 = 256
max_number_of_tracks_log_2 = 8
batch_size = 50
eps = 0.15

z0_file = "/media/lucas/QS/binaries-trk/OldKF_TTbar_170K_quality-1-trk-z0.bin"
pt_file = "/media/lucas/QS/binaries-trk/OldKF_TTbar_170K_quality-1-trk-pt.bin"
z0 = np.fromfile(z0_file, dtype=np.float32)
pt = np.fromfile(pt_file, dtype=np.float32)

from batched_dbscan import BatchedDBSCAN

db = BatchedDBSCAN(z0, pt, eps, batch_size, max_number_of_tracks, False, False)

In [47]:
np.set_printoptions(precision=2)
np.set_printoptions(suppress=True)

start_idx = 0
end_idx = start_idx + db.batch_size

In [48]:
n_pad = (db.n_batches * db.batch_size) - db.z0.shape[0]

In [49]:
db.z0 = db.pad_vector(db.z0, n_pad, 21)
db.pt = db.pad_vector(db.pt, n_pad, 0)

In [99]:
db.z0_batches = {}
db.pt_batches = {}
db.rs_batches = {}
db.left_boundaries_batches = {}
db.boundaries_batches = {}
db.clusters_batches = {}

In [100]:
pv_cluster = np.zeros((1, 4))
merge_count = 0

**fitloop**

In [105]:
clusters = db.initialize_clusters(db.max_n_tracks_batched)
for i in range(db.n_batches):
    print(i)
    start_idx = i * db.batch_size
    end_idx = (i + 1) * db.batch_size
    z0_batch = db.z0[start_idx:end_idx]
    pt_batch = db.pt[start_idx:end_idx]

    track_batch = db.build_tracks(z0_batch, pt_batch)

    db.tracks = track_batch

    rs_batch = db.pad_vector(
        track_batch[:, 1], db.batch_size_power_2 - db.batch_size, 0
    )
    rs_batch = db.prefix_sum(rs_batch)

    db.rs = rs_batch

    # Storing batches
    db.z0_batches[i] = track_batch[:, 0]
    db.pt_batches[i] = track_batch[:, 1]
    db.rs_batches[i] = rs_batch

    # Finding Left Boundaries
    left_boundaries = db.find_left_boundaries(track_batch)
    db.left_boundaries_batches[i] = left_boundaries

    # Finding Right Boundaries
    boundaries = db.find_right_boundaries(left_boundaries, rs_batch, track_batch)
    db.boundaries_batches[i] = boundaries
    db.boundaries = boundaries
    clusters_batch = db.convert_boundaries_to_clusters(boundaries)
    clusters[i * db.batch_size : (i + 1) * db.batch_size, :] = clusters_batch

    db.results[i] = clusters_batch


clusters = db.merge_clusters(clusters)
db.clusters = clusters

# find pv_clusters
pv_cluster[0, :] = clusters.iloc[db.max_pt_i, :]
print(db.max_pt, db.max_pt_i)
print(f"Merged count: {db.merge_count}")
pv_tracks = []

for i in range(db.max_number_of_tracks):
    z0_trk = db.z0[i]

    if (z0_trk >= pv_cluster[0, db.z0_low_idx]) and (
        z0_trk <= pv_cluster[0, db.z0_high_idx]
    ):
        pv_tracks.append(z0_trk)
median_vertex = db.get_vertex(np.array(pv_tracks))
db.z0_pv = np.median(pv_tracks)

0
1
2
3
4
142.0233987569809 107
Merged count: 42


In [106]:
db.z0_pv

-2.2265625

In [96]:
db.merged_list

['[-5.51, 7.96, -5.62] + [-5.1, 2.7, -5.1] -> [-5.51, 7.96, -10.72]',
 '[-5.51, 7.96, -10.72] + [-4.92, 4.85, -4.92] -> [-5.51, 7.96, -15.64]',
 '[-5.51, 7.96, -15.64] + [-4.69, 1.96, -4.69] -> [-5.51, 7.96, -20.33]',
 '[-5.51, 7.96, -20.33] + [-4.22, 2.35, -4.22] -> [-5.51, 7.96, -24.55]',
 '[-5.51, 7.96, -24.55] + [-4.04, 4.33, -4.04] -> [-5.51, 7.96, -28.59]',
 '[-5.51, 7.96, -28.59] + [-3.52, 25.09, -3.69] -> [-5.51, 25.09, -32.29]',
 '[-5.51, 25.09, -32.29] + [-2.93, 6.52, -3.05] -> [-5.51, 25.09, -35.33]',
 '[-5.51, 25.09, -35.33] + [-2.58, 2.09, -2.58] -> [-5.51, 25.09, -37.91]',
 '[-5.51, 25.09, -37.91] + [-1.88, 42.53, -2.4] -> [-5.51, 42.53, -40.31]',
 '[-5.51, 42.53, -40.31] + [-1.7, 2.71, -1.7] -> [-5.51, 42.53, -42.01]',
 '[-5.51, 42.53, -42.01] + [-1.41, 6.9, -1.46] -> [-5.51, 42.53, -43.48]',
 '[-5.51, 42.53, -43.48] + [-0.94, 2.9, -0.94] -> [-5.51, 42.53, -44.41]',
 '[-5.51, 42.53, -44.41] + [-0.76, 2.09, -0.76] -> [-5.51, 42.53, -45.18]',
 '[-5.51, 42.53, -45.18] + [-0

In [86]:
db.max_pt

142.0233987569809

In [87]:
db.max_pt_i

107

In [89]:
clusters.iloc[db.max_pt_i]

pt_sum     142.02
z0_low      -3.16
z0_high     -1.41
noise        0.00
Name: 107, dtype: float64

In [85]:
clusters.sort_values(by="pt_sum", ascending=False)

,pt_sum,z0_low,z0_high,noise
107,142.02,-3.16,-1.41,0.0
102,104.91,-3.81,-3.40,0.0
111,51.44,-0.12,0.41,0.0
109,17.67,-0.59,-0.29,0.0
118,17.04,3.46,3.63,0.0
...,...,...,...,...
87,0.00,21.00,21.00,0.0
88,0.00,21.00,21.00,0.0
89,0.00,21.00,21.00,0.0
90,0.00,21.00,21.00,0.0


In [31]:
left_boundaries.astype(int)

array([1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1])

In [32]:
track_batch[:, 0]

array([-5.62, -5.51, -5.1 , -4.92, -4.92, -4.69, -4.22, -4.04, -4.04,
       -3.69, -3.69, -3.63, -3.57, -3.52, -3.05, -2.93, -2.93, -2.58,
       -2.4 , -2.34, -2.23, -2.23, -2.23, -2.17, -2.05, -1.99, -1.88,
       -1.7 , -1.46, -1.41, -0.94, -0.76, -0.41, -0.29,  0.06,  0.23,
        0.23,  0.29,  0.41,  0.88,  1.35,  2.11,  2.81,  3.57,  3.87,
        5.1 ,  5.74,  6.5 ,  6.62,  7.03])

In [33]:
np.diff(track_batch[:, 0])

array([0.12, 0.41, 0.18, 0.  , 0.23, 0.47, 0.18, 0.  , 0.35, 0.  , 0.06,
       0.06, 0.06, 0.47, 0.12, 0.  , 0.35, 0.18, 0.06, 0.12, 0.  , 0.  ,
       0.06, 0.12, 0.06, 0.12, 0.18, 0.23, 0.06, 0.47, 0.18, 0.35, 0.12,
       0.35, 0.18, 0.  , 0.06, 0.12, 0.47, 0.47, 0.76, 0.7 , 0.76, 0.29,
       1.23, 0.64, 0.76, 0.12, 0.41])

In [34]:
a = pd.DataFrame({"z0": track_batch[:, 0]})

In [35]:
a["diff"] = a.diff()

In [36]:
a["label"] = left_boundaries.astype(int)

In [37]:
a

,z0,diff,label
0,-5.62,NaN,1
1,-5.51,0.12,0
2,-5.10,0.41,1
3,-4.92,0.18,1
4,-4.92,0.00,0
5,-4.69,0.23,1
6,-4.22,0.47,1
7,-4.04,0.18,1
8,-4.04,0.00,0
9,-3.69,0.35,1


In [38]:
track_batch

array([[-5.62,  4.83,  0.  ],
       [-5.51,  3.13,  0.  ],
       [-5.1 ,  2.7 , -1.  ],
       [-4.92,  2.89, -1.  ],
       [-4.92,  1.96,  0.  ],
       [-4.69,  1.96, -1.  ],
       [-4.22,  2.35, -1.  ],
       [-4.04,  2.21, -1.  ],
       [-4.04,  2.12,  0.  ],
       [-3.69,  1.98, -1.  ],
       [-3.69,  1.98,  0.  ],
       [-3.63,  5.01,  0.  ],
       [-3.57,  9.02,  0.  ],
       [-3.52,  7.1 ,  0.  ],
       [-3.05,  2.11, -1.  ],
       [-2.93,  2.43,  0.  ],
       [-2.93,  1.97,  0.  ],
       [-2.58,  2.09, -1.  ],
       [-2.4 ,  2.8 , -1.  ],
       [-2.34,  3.1 ,  0.  ],
       [-2.23,  4.83,  0.  ],
       [-2.23,  7.7 ,  0.  ],
       [-2.23,  4.24,  0.  ],
       [-2.17,  2.15,  0.  ],
       [-2.05, 10.22,  0.  ],
       [-1.99,  5.04,  0.  ],
       [-1.88,  2.46,  0.  ],
       [-1.7 ,  2.71, -1.  ],
       [-1.46,  4.44, -1.  ],
       [-1.41,  2.46,  0.  ],
       [-0.94,  2.9 , -1.  ],
       [-0.76,  2.09, -1.  ],
       [-0.41,  4.1 , -1.  ],
       [-0

In [39]:
boundaries

array([[  0.  ,   0.  ,   4.83,   4.83,  -5.62,  -5.51,   0.  ],
       [  1.  ,   4.83,   7.96,   3.13,  -5.51,  -5.1 ,   0.  ],
       [  2.  ,   7.96,  10.66,   2.7 ,  -5.1 ,  -5.1 ,   1.  ],
       [  3.  ,  10.66,  13.55,   2.89,  -4.92,  -4.92,   0.  ],
       [  4.  ,  13.55,  15.51,   1.96,  -4.92,  -4.69,   0.  ],
       [  5.  ,  15.51,  17.47,   1.96,  -4.69,  -4.69,   1.  ],
       [  6.  ,  17.47,  19.82,   2.35,  -4.22,  -4.22,   1.  ],
       [  7.  ,  19.82,  22.03,   2.21,  -4.04,  -4.04,   0.  ],
       [  8.  ,  22.03,  24.15,   2.12,  -4.04,  -3.69,   0.  ],
       [  9.  ,  24.15,  26.12,   1.98,  -3.69,  -3.69,   0.  ],
       [ 13.  ,  42.14,  49.23,   7.1 ,  -3.52,  -3.05,   0.  ],
       [ 14.  ,  49.23,  51.34,   2.11,  -3.05,  -2.93,   0.  ],
       [ 16.  ,  53.77,  55.75,   1.97,  -2.93,  -2.58,   0.  ],
       [ 17.  ,  55.75,  57.84,   2.09,  -2.58,  -2.58,   1.  ],
       [ 18.  ,  57.84,  60.63,   2.8 ,  -2.4 ,  -2.34,   0.  ],
       [ 26.  ,  97.91, 1

In [44]:
boundaries.shape

(50, 7)

**SEEMS LIKE MY BOUNDARIES ARE FINE!**

In [55]:
clusters_batch.shape

(50, 4)

In [56]:
clusters_batch

array([[-5.62, -5.51,  7.96,  0.  ],
       [-5.1 , -5.1 ,  2.7 ,  1.  ],
       [-4.92, -4.92,  4.85,  0.  ],
       [-4.69, -4.69,  1.96,  1.  ],
       [-4.22, -4.22,  2.35,  1.  ],
       [-4.04, -4.04,  4.33,  0.  ],
       [-3.69, -3.52, 25.09,  0.  ],
       [-3.05, -2.93,  6.52,  0.  ],
       [-2.58, -2.58,  2.09,  1.  ],
       [-2.4 , -1.88, 42.53,  0.  ],
       [-1.7 , -1.7 ,  2.71,  1.  ],
       [-1.46, -1.41,  6.9 ,  0.  ],
       [-0.94, -0.94,  2.9 ,  1.  ],
       [-0.76, -0.76,  2.09,  1.  ],
       [-0.41, -0.29,  6.25,  0.  ],
       [ 0.06,  0.06,  1.97,  1.  ],
       [ 0.23,  0.41, 10.61,  0.  ],
       [ 0.88,  0.88,  4.87,  1.  ],
       [ 1.35,  1.35,  2.79,  1.  ],
       [ 2.11,  2.11,  4.8 ,  1.  ],
       [ 2.81,  2.81,  3.19,  1.  ],
       [ 3.57,  3.57,  3.3 ,  1.  ],
       [ 3.87,  3.87,  2.18,  1.  ],
       [ 5.1 ,  5.1 ,  2.54,  1.  ],
       [ 5.74,  5.74,  2.31,  1.  ],
       [ 6.5 ,  6.62,  4.15,  0.  ],
       [ 7.03,  7.03,  2.06,  1.  ],
 

**clusters seem fine**

maybe clusters initialized with a 0 value might be an issue? 

well no, actually... the cluster_batch should not be of length 250. it should be of length batch_size?? right? 
boundaries are of the correct size

ok so i fixed that. clusters in `convert_boundaries_to_clusters` will be of the same size as the `n_boundaries`

**What are the clusters/ boundaries from dbscan ?**

In [16]:
dfb = pd.DataFrame({"z0": z0_batch, "pt": pt_batch})

In [17]:
from sklearn.cluster import DBSCAN

In [18]:
_db = DBSCAN(eps=0.15, min_samples=2).fit(dfb["z0"].values.reshape(-1, 1))

In [19]:
dfb["label"] = _db.labels_

In [20]:
max_label = dfb.label.max()

In [21]:
n_noise = dfb[dfb.label == -1].shape[0]

In [22]:
dfb.loc[dfb.label == -1, "label"] = np.arange(n_noise) + max_label + 1

In [23]:
clusters_batch = dfb.groupby(["label"]).agg(
    {"z0": [np.min, np.max], "pt": [np.sum, "count"]}
)

In [24]:
clusters_batch.columns = ["z0_low", "z0_high", "pt_sum", "ntracks"]

In [25]:
clusters_batch["noise"] = 0

In [26]:
clusters_batch.loc[clusters_batch.ntracks == 1, "noise"] = 1

In [27]:
clusters_batch.drop(columns=["ntracks"], inplace=True)

In [28]:
pd.set_option("display.precision", 2)

In [29]:
clusters_batch.sort_values(by=["z0_low"])

,z0_low,z0_high,pt_sum,noise
label,,,,
6,-5.62,-5.51,7.96,0
14,-5.10,-5.10,2.70,1
8,-4.92,-4.92,4.85,0
20,-4.69,-4.69,1.96,1
12,-4.22,-4.22,2.35,1
0,-4.04,-4.04,4.33,0
4,-3.69,-3.52,25.09,0
5,-3.05,-2.93,6.52,0
10,-2.58,-2.58,2.09,1
